In [8]:
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [9]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TerraDataset(Dataset):


    def __init__(self, file):
        self.file = file
        self.strings = self.load()
        self.tokens = self.transform()

    def load(self):
      data = [json.loads(d) for d in open(self.file, encoding="utf-8")]
      s1 = []
      for a in data:
        s1.append( a['premise']+' ~> '+a['hypothesis'])
      return s1


    def transform(self):
      model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
      tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
      input_ids = []
      for string in self.strings:
        input_ids.append(tokenizer(string))
      return input_ids


    def __len__(self):
        return len(self.strings)


    def __getitem__(self, idx):
        return self.tokens[idx]

In [10]:
from transformers import DataCollatorForLanguageModeling
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [11]:
training_data = TerraDataset('/content/drive/MyDrive/Colab Notebooks/TERRa/train.jsonl')
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True, collate_fn = collator)

In [12]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [35]:
l = []
for batch_ndx, sample in enumerate(train_dataloader):
    for i in range(8):
      l.append(len(sample['input_ids'][i][sample['input_ids'][i]<50257]))
    a = model(input_ids = sample['input_ids'],attention_mask = sample['attention_mask'],return_dict = True,output_hidden_states = True)['hidden_states'][-1]
    if batch_ndx == 0:
      break

In [36]:
for i in range(len(l)):
  print(a[i][:l[i]])
  print(len(a[i][:l[i]]))

tensor([[-0.2270,  0.6160, -1.5585,  ...,  0.4662, -0.2358,  0.0687],
        [ 0.7264,  0.4682,  0.2207,  ..., -0.1492, -0.4717,  0.4951],
        [ 0.4565,  0.9379, -1.8596,  ..., -1.3401, -0.1594, -0.9761],
        ...,
        [-0.1426,  1.0398,  0.2604,  ...,  1.6875, -0.7068,  0.3179],
        [ 1.1666, -0.1701,  0.2802,  ..., -0.0688, -1.1162,  0.8281],
        [ 0.1356,  0.0768, -0.9397,  ...,  0.1016, -0.2293,  0.1786]],
       grad_fn=<SliceBackward>)
22
tensor([[ 0.4087,  0.6969, -1.1862,  ...,  0.7027, -0.2365, -0.2996],
        [ 0.4895,  1.4049, -0.9997,  ...,  2.4376, -1.5799,  0.4110],
        [-0.6061,  1.2796, -0.1095,  ...,  0.9793,  0.0112, -0.3486],
        ...,
        [-1.4037,  1.2565,  1.4036,  ...,  1.4117, -0.4222,  0.2294],
        [-0.5907, -0.0746,  0.2566,  ..., -0.9806, -1.3750,  0.7238],
        [ 0.6737, -0.3793, -0.5567,  ..., -0.9508, -0.1273,  1.1467]],
       grad_fn=<SliceBackward>)
106
tensor([[-0.1411,  0.4318, -1.5266,  ...,  0.3838,  0.0608, -